In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
! pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 52.8MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# !pip install gensim

In [ ]:
# !pip install fasttext

In [ ]:
# ! pip install senticnet

In [ ]:
# from senticnet.babelsenticnet import BabelSenticNet

In [ ]:
# sn = BabelSenticNet('kr')
# sn.concept('사랑')

In [ ]:
# from sklearn.utils import class_weight
# class_weights_fci = class_weight.compute_class_weight('balanced', np.unique(data_ver2['Emotion']), data_ver2['Emotion'])

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, GRU, Conv1D,Flatten,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Bidirectional

Using TensorFlow backend.


In [ ]:
# def featurize_rnn(corpus,wdim,maxlen):
#     rnn_total = np.zeros((len(corpus),maxlen,wdim))
#     for i in range(len(corpus)):
#         if i%1000 ==0:
#             print(i)
#         s = corpus[i]
#         for j in range(len(s)):
#             if s[-j-1] in data_ver2['Sentence'] and j < maxlen:
#                 rnn_total[i][-j-1,:] = model_ft[s[-j-1]]
#     return rnn_total

# data = featurize_rnn(tokenized_data,100,30)

# from keras.layers import LSTM
# from keras.layers import Bidirectional

# def validate_bilstm(result,y,hidden_lstm,hidden_dim,cw,filename):
#     model = Sequential()
#     model.add(Bidirectional(LSTM(hidden_lstm), input_shape=(len(result[0]), len(result[0][0]))))
#     model.add(layers.Dense(hidden_dim, activation='relu'))
#     model.add(layers.Dense(int(max(y)+1), activation='softmax'))
#     model.summary()
#     model.compile(optimizer='Adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#     filepath=filename+"-{epoch:02d}-{val_acc:.4f}.hdf5"
#     checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')
#     callbacks_list = [metricsf1macro,checkpoint]
#     model.fit(result,y,validation_split=0.1,epochs=30,batch_size=16,callbacks=callbacks_list,class_weight=cw)

# validate_bilstm(data,data_ver2['Emotion'],32,128,class_weights_fci,'model/tutorial/rec')

In [ ]:
# for t in tokenized_data[0]:
#   print(t)
#   try:
#     print(sn.concept(t))
#   except:
#     continue

In [ ]:
#! git clone https://github.com/facebookresearch/fastText.git
# ! cd fastText
# ! sudo python setup.py install

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
data_ver1 = pd.read_csv('train.csv')
data_ver2= pd.read_csv('train2.csv')
data_ver2.drop(list(range(204,505)), inplace= True)
data_ver2

,Sentence,Emotion
0,그 사람이 죽어버렸으면 좋겠어요,0
1,오늘 진짜 짜증나,0
2,맛집이라 해서 갔는데 맛이 쓰레기야,0
3,과제가 너무 많아,0
4,오늘도 야근이야,0
...,...,...
582,나아지고 있는 거 같아,3
583,주말이야,3
584,너무 고마워,3
585,결혼할 거 같아,3


In [ ]:
(data_ver2[data_ver2['Emotion']==1])#389 -> 90
#len(data_ver2[data_ver2['Emotion']==0])#77
#len(data_ver2[data_ver2['Emotion']==2])#31
#len(data_ver2[data_ver2['Emotion']==3])#90



,Sentence,Emotion
10,술 먹고 싶어요,1
11,죽고 싶어,1
12,울고 싶어,1
13,노력해서 되는 일이 있을까?,1
14,너무 지친다,1
...,...,...
199,끝나니까 허무하다,1
200,꿈이 이뤄질까,1
201,내가 너무 소심한가,1
202,말실수 한거 같아,1


In [ ]:
okt = Okt()
stopwords = ['백주부','ㅋ','ㅎ','ㅠㅠ','ㅜㅜ','^^','의','을','(','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', 'ㄴ','ㅠ',',','0',';','ㅜ','/','~','!','?','♥♥','♡♡','♡','♥']


In [ ]:
  def tokenize(data):

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    han = re.compile(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")

    arr  =[]
    for sen in data:
      sen = re.sub(emoji_pattern, "", sen)
      sen = re.sub(han, "", sen)
      temp = okt.morphs(sen, stem = True)#True
      temp = [word for word in temp if not word in stopwords]
      arr.append(' '.join(temp))
      #arr.append(temp)
    return arr

In [ ]:
tokenized_data = tokenize(data_ver1['Sentence'])

In [ ]:
#tokenized_data1 = tokenize(data_ver1['Sentence'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import accuracy_score

In [ ]:
#print(len(data_ver1))

In [ ]:
# vectorize = TfidfVectorizer(max_features = 5000)
# vectorized_data = vectorize.fit_transform(tokenized_data)

In [ ]:
train_D = (tokenized_data)
train_L = data_ver1['Emotion']

In [ ]:
train_D, test_D,train_L, test_L = train_test_split(tokenized_data, data_ver1['Emotion'].values , random_state =42, train_size= 0.8, shuffle = True)

In [ ]:
#train_D2, test_D2,train_L2, test_L2 = train_test_split(tokenized_data1, data_ver1['Emotion'].values , random_state =42, train_size= 0.8, shuffle = True)

In [ ]:
# lgbm = lightgbm.LGBMClassifier()

In [ ]:
# lgbm.fit(train_D, train_L)
# pred = lgbm.predict(test_D)

In [ ]:
# accuracy_score(pred, test_L)

In [ ]:
# def featurize_rnn(corpus,wdim,maxlen):
#     rnn_total = np.zeros((len(corpus),maxlen,wdim))
#     for i in range(len(corpus)):
#         if i%1000 ==0:
#             print(i)
#         s = corpus[i]
#         for j in range(len(s)):
#             if s[-j-1] in model_ft and j < maxlen:
#                 rnn_total[i][-j-1,:] = model_ft[s[-j-1]]
#     return rnn_total

# #fci_rec = featurize_rnn(train_D + test_D,100,7)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
train_L = one.fit_transform(np.array(train_L).reshape(-1,1)).toarray()
test_L = one.transform(np.array(test_L).reshape(-1,1)).toarray()
#train_L2 = one.transform(np.array(train_L2).reshape(-1,1)).toarray()


In [ ]:
# model = Sequential()
# model.add(Embedding(5000, 100))#, trainable = True))
# model.add(Bidirectional(LSTM(128, activation = 'tanh')))
# model.add(Dropout(0.5))
# model.add(Dense(128))
# model.add(Dropout(0.5))
# model.add(Dense(train_L.shape[1], activation='softmax'))

In [ ]:
import tensorflow_hub as hub 
import tensorflow as tf
from keras import backend as K
import urllib.request


In [ ]:
# ! pip uninstall tensorflow tensorflow_hub tensorflowjs
# ! pip install tensorflow==1.15 tensorflow_hub==0.5.0 tensorflowjs==1.2.6

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
def ELMoEmbedding(x):
  return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict = True, signature = 'default')['default']

In [ ]:
from keras.callbacks import TensorBoard#ModelCheckpoint
##es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)
#mc = ModelCheckpoint('best_t_elmo_model_onlytrain.h5', monitor='val_acc', mode='max', verbose=1, save_weights_only=False)

In [ ]:
# class ElmoEmbeddingLayer(tf.keras.layers.Layer):
#     """Taken from: 
#     https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb"""
#     def __init__(self, **kwargs):
#         self.dimensions = 1024
#         self.trainable=False
#         super(ElmoEmbeddingLayer, self).__init__(**kwargs)
#     def build(self, input_shape):
#         self.elmo = hub.Module(
#             'https://tfhub.dev/google/elmo/1', 
#             trainable=self.trainable,
#             name="{}_module".format(self.name)
#         )
#         # Changed assuming trainable weights might be set using 
#         super(ElmoEmbeddingLayer, self).build(input_shape)
#     def call(self, x, mask=None):
#         result = self.elmo(
#             K.squeeze(K.cast(x, tf.string), axis=1),
#             as_dict=True,
#             signature='default',
#         )['default']
#         return result
#     def compute_mask(self, inputs, mask=None):
#         return K.not_equal(inputs, '--PAD--')
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.dimensions)

In [ ]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input, Dropout
from keras import optimizers
# model = Sequential()
# model.add(tf.keras.layers.InputLayer(dtype=  'string', input_shape = (1,)))
# model.add(ElmoEmbeddingLayer())
# model.add(tf.keras.layers.Dense(256, activation = 'relu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(4, activation=  'softmax'))
input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
dropout = Dropout(0.5)(hidden_layer)
output_layer = Dense(4, activation='softmax')(dropout)
model = Model(inputs=[input_text], outputs=output_layer)
# sess = tf.Session()
# K.set_session(sess)
# sess.run(tf.global_variables_initializer())
# sess.run(tf.tables_initializer())
adam = optimizers.Adam(clipvalue = 0.6,  learning_rate = 1e-3)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])#'categorical_crossentropy'

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
import datetime
log_dir= 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorboard_callback = TensorBoard(log_dir= log_dir, histogram_freq = 1)

In [ ]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 1028      
Total params: 263,428
Trainable params: 263,428
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(np.asarray(train_D), np.asarray(train_L),epochs = 20,batch_size=  30,validation_split=  0.2)#, validation_data = (np.array(test_D), np.array(test_L)))

Train on 13951 samples, validate on 3488 samples
Epoch 1/20
13950/13951 [============================>.] - ETA: 0s - loss: 1.3786 - acc: 0.3035

InvalidArgumentError: ignored

In [ ]:
import json



with open('elmo_data.json', 'w') as f:
    json.dump(str(history.history), f)



In [ ]:
with open('elmo_data.json', 'r') as f:
  d = json.load(f)
print(d)

{'val_loss': [1.2872377336025238, 1.2467638552188873, 1.1991693675518036, 1.2291805446147919, 1.2139660716056824, 1.2411887645721436, 1.2541942596435547, 1.3014951646327972, 1.2422834634780884, 1.3597261905670166, 1.2623080909252167, 1.2978671789169312, 1.288278877735138, 1.2837488949298859, 1.3434213995933533, 1.338758796453476, 1.371456891298294, 1.4080892503261566, 1.383622407913208, 1.4326256811618805], 'val_acc': [0.30000001192092896, 0.375, 0.44999998807907104, 0.375, 0.44999998807907104, 0.42500001192092896, 0.42500001192092896, 0.44999998807907104, 0.42500001192092896, 0.4000000059604645, 0.4749999940395355, 0.4749999940395355, 0.4749999940395355, 0.4749999940395355, 0.4749999940395355, 0.4749999940395355, 0.5, 0.44999998807907104, 0.4749999940395355, 0.4749999940395355], 'loss': [1.5479077845811844, 1.2239396125078201, 1.1383262798190117, 0.9752128422260284, 0.8443870432674885, 0.7926311455667019, 0.6706805564463139, 0.6317084934562445, 0.575370866805315, 0.5424670688807964, 0

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
import time
s = time.time()
model.evaluate(np.asarray(test_D), np.array(test_L))
#print(time.time() - s)

58/58 [==============================] - 0s 1ms/step


[1.5901973288634728, 0.36206895112991333]

In [ ]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
dropout = Dropout(0.5)(hidden_layer)
output_layer = Dense(4, activation='softmax')(dropout)
loaded_model = Model(inputs=[input_text], outputs=output_layer)
adam = optimizers.Adam(clipvalue = 0.6,  learning_rate = 1e-3)
loaded_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
from keras.models import load_model
model.save_weights('elmo_weight.h5')


In [ ]:
import time
s = time.time()
loaded_model.load_weights('elmo_weight.h5')
loaded_model.evaluate(np.asarray(test_D), np.array(test_L))
#print(time.time() - s)

58/58 [==============================] - 0s 1ms/step


[1.5901973288634728, 0.36206895112991333]

In [ ]:
#loaded_model.evaluate(np.asarray(test_D), np.array(test_L))

In [ ]:
# ! pip uninstall tensorflow
# ! pip install tensorflow== 2.0

In [ ]:
model.save('elmo.h5')

In [ ]:
import json
model_json = model.to_json()
with open("elmo_model.json", "w") as json_file : 
    json_file.write(model_json)

In [ ]:
load_model('elmo_weight.h5')

In [ ]:
def sentiment_predict(new_sentence):
  # new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  # new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  new_sentence = tokenize(new_sentence)
  #new_sentence = ' '.join(new_sentence)
  #new_sentence = [word for word in ' '.join(new_sentence)]
  print(new_sentence)

  pred= model.predict(np.asarray(new_sentence)) # 예측
  return pred

In [ ]:
sentiment_predict(['dP', ''])

['개 빡치다 시발 진짜', '']


array([[0.7442806 , 0.16920544, 0.02762927, 0.05888473],
       [0.2507515 , 0.2503804 , 0.24899194, 0.24987617]], dtype=float32)